In [1]:
import logging

log_file_name = 'decision.log'
log_mode='w'
log_level = logging.DEBUG

logger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_file_name, mode=log_mode)
formatter = logging.Formatter('%(asctime)s - %(module)s - %(funcName)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(log_level)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
def print_log(log_file_name):
    with open(log_file_name, 'r') as log_file:
        for line in log_file:
            print line.strip()

In [4]:
from person import Person
from externalities import World, Categorical, Offer

In [5]:
world = World()
print_log(log_file_name)

2017-07-21 13:29:53,532 - externalities - __init__ - root - INFO - World initialized


In [6]:
Offer.print_help()

timestamp: int between 0 and 999999999
valid_from: int between 0 and 999999999
valid_to: int between 0 and 999999999
difficulty: positive numeric
reward: positive numeric
channel: Categorical(['web' 'email' 'mobile' 'social'])
type: Categorical(['bogo' 'discount' 'informational'])


In [7]:
offer_channel = Categorical(('web', 'email', 'mobile', 'social'), (1, 1, 1, 1))
offer_type = Categorical(('bogo', 'discount', 'informational'), (0, 1, 0))

discount = Offer(0, valid_from=10, valid_until=20, difficulty=10, reward=2, channel=offer_channel, offer_type=offer_type)

In [9]:
Person.print_help()

id: uuid.uuid4() (set automatically)
dob: 'YYYYMMDD'
gender: ['M', 'F', 'O']
became_member_on: 'YYYYMMDD'
income: positive numeric
taste: Categorical(['sweet','sour','salty','bitter','umami'])
marketing_segment: Categorical(['front page','local','entertainment','sports','opinion','comics'])

last_transaction: Transaction
last_unviewed_offer: Offer
last_viewed_offer: Offer
history: list of Offers and Transactions sorted by Event timestamps

view_offer_sensitivity: Categorical(['background','offer_age','web','email','mobile','social'])
make_purchase_sensitivity: Categorical(['background','time_since_last_transaction','last_viewed_offer_strength','viewed_active_offer'])
purchase_amount_sensitivity: Categorical(['background','income_adjusted_purchase_sensitivity','front page','local','entertainment','sports','opinion','comics','sweet','sour','salty','bitter','umami'])


In [10]:
person_taste = Categorical(['sweet','sour','salty','bitter','umami'],
                           [1,1,0,0,0])

person_marketing_segment = Categorical(['front page','local','entertainment','sports','opinion','comics'],
                                       [1, 1, 0, 0, 0, 1])

person_view_offer_sensitivity = Categorical(['background', 'offer_age','web','email','mobile', 'social'],
                                            [0, -1, 1, 1, 1, 1])

person_make_purchase_sensitivity = Categorical(['background', 'time_since_last_transaction','time_since_last_viewed_offer','viewed_active_offer','web','email','mobile'])
person_purchase_amount_sensitivity = Categorical(['background','income_adjusted_purchase_sensitivity','front page','local','entertainment','sports','opinion','comics','sweet','sour','salty','bitter','umami'])

person = Person(became_member_on='20170716', view_offer_sensitivity=person_view_offer_sensitivity)
person.last_unviewed_offer = discount

In [11]:
discount.__dict__

{'channel': <externalities.Categorical at 0x9a08160>,
 'completed': False,
 'difficulty': 10,
 'id': 'a2e2c943bd07453e8a8100c48c9a87b1',
 'offer_type': <externalities.Categorical at 0x9a081d0>,
 'progress': 0.0,
 'reward': 2,
 'timestamp': 0,
 'type': 'offer',
 'valid_from': 10,
 'valid_until': 20}

In [12]:
person.__dict__

{'became_member_on': '20170716',
 'dob': '19010101',
 'dt_fmt': '%Y%m%d',
 'gender': None,
 'history': [],
 'id': 'b9c0c2aa31b0443e8c93a56e26b29ac1',
 'income': None,
 'last_transaction': None,
 'last_unviewed_offer': <externalities.Offer at 0x9a080b8>,
 'last_viewed_offer': None,
 'make_purchase_sensitivity': <externalities.Categorical at 0x9a09128>,
 'marketing_segment': <externalities.Categorical at 0x9a08f28>,
 'purchase_amount_sensitivity': <externalities.Categorical at 0x9a09080>,
 'taste': <externalities.Categorical at 0x9a08f60>,
 'view_offer_sensitivity': <externalities.Categorical at 0x9a08f98>}

In [13]:
def g(x):
    return 1.0/(1.0 + np.exp(-x))

def g_inv(y):
    return np.log(y/(1.0-y))

In [14]:
g_inv(0.01)/float(1*24*30)

-0.0063821109029647081

In [15]:
for i in range(-5,5):
    print i, g(i+1) - g(i)

-5 0.0112933590378
-4 0.0294396632155
-3 0.0717770488446
-2 0.149738499348
-1 0.23105857863
0 0.23105857863
1 0.149738499348
2 0.0717770488446
3 0.0294396632155
4 0.0112933590378


In [27]:
offer_channel = Categorical(('web', 'email', 'mobile', 'social'), 
                            (1, 1, 1, 1))
offer_type = Categorical(('bogo', 'discount', 'informational'), 
                         (0, 1, 0))

discount = Offer(0, valid_from=10, valid_until=20, difficulty=10, reward=2, channel=offer_channel, offer_type=offer_type)

person_view_offer_sensitivity = Categorical(['background', 
                                             'offer_age',
                                             'web','email','mobile', 'social'],
                                            [g_inv(0.20) - 4, 
                                             -abs(g_inv(0.01)/float(1*24*30)), 
                                             0, 1, 1, 2])

person_make_purchase_sensitivity = Categorical(['background', 
                                                'time_since_last_transaction',
                                                'last_viewed_offer_strength',
                                                'viewed_active_offer'],
                                               [g_inv(1.0/24.0),
                                                abs(g_inv(0.10)/float(1*24*30)),
                                                1,
                                                1])

person_purchase_amount_sensitivity = Categorical(['background',
                                                  'income_adjusted_purchase_sensitivity',
                                                  'front page',
                                                  'local',
                                                  'entertainment',
                                                  'sports',
                                                  'opinion',
                                                  'comics',
                                                  'sweet',
                                                  'sour',
                                                  'salty',
                                                  'bitter',
                                                  'umami'],
                                                [1,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0,
                                                 0])

person = Person(became_member_on='20170716', 
                view_offer_sensitivity=person_view_offer_sensitivity,
                make_purchase_sensitivity=person_make_purchase_sensitivity,
                purchase_amount_sensitivity=person_purchase_amount_sensitivity)
person.last_viewed_offer = discount
print 'Offer Viewed'

world.world_time = 0


history = list()
for i in range(24*30):
    world.world_time = i
    transaction = person.make_purchase(world)
    if transaction:
        history.append(1)
    else:
        history.append(0)

print len(history), sum(history), np.mean(history)

Offer Viewed
720 32 0.0444444444444


In [23]:
transaction.__dict__

{'amount': 1.81, 'timestamp': 4}

In [16]:
print_log(log_file_name)

2017-07-16 17:59:22,361 - externalities - __init__ - root - INFO - World initialized
2017-07-16 17:59:22,400 - person - __init__ - root - INFO - Person initialized
2017-07-16 17:59:22,430 - person - __init__ - root - INFO - Person initialized
2017-07-16 17:59:22,566 - person - __init__ - root - INFO - Person initialized
2017-07-16 17:59:22,569 - person - make_purchase - root - DEBUG - Made purchase decision at time t = 0
2017-07-16 17:59:22,575 - person - make_purchase - root - DEBUG -         beta = [-2.7080502  0.0030517  1.         1.       ]
2017-07-16 17:59:22,575 - person - make_purchase - root - DEBUG -            x = [ 1.  0.  1.  0.]
2017-07-16 17:59:22,575 - person - make_purchase - root - DEBUG -       beta*x = [-2.7080502  0.         1.         0.       ]
2017-07-16 17:59:22,575 - person - make_purchase - root - DEBUG - dot(beta, x) = -1.7080502011
2017-07-16 17:59:22,575 - person - make_purchase - root - DEBUG -            p = 0.153416784696
2017-07-16 17:59:22,575 - perso

In [17]:
g(-3.708 + 1)

0.062502941535436882

In [18]:
1.0/16.0

0.0625

In [19]:
g_inv(0.01)/float(1*24*30)

-0.0063821109029647081